<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Redis-in-pystore" data-toc-modified-id="Redis-in-pystore-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Redis in pystore</a></span><ul class="toc-item"><li><span><a href="#Making-a-store" data-toc-modified-id="Making-a-store-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Making a store</a></span></li><li><span><a href="#The-normal-case-is-a-string/bytes" data-toc-modified-id="The-normal-case-is-a-string/bytes-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>The normal case is a string/bytes</a></span></li><li><span><a href="#Lists" data-toc-modified-id="Lists-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Lists</a></span></li></ul></li><li><span><a href="#Using-Redis-to-operate-on-streams" data-toc-modified-id="Using-Redis-to-operate-on-streams-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Using Redis to operate on streams</a></span></li><li><span><a href="#Ignore-everything-below----it's-just-scrap" data-toc-modified-id="Ignore-everything-below----it's-just-scrap-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Ignore everything below -- it's just scrap</a></span><ul class="toc-item"><li><span><a href="#Scrap-for-RedisList" data-toc-modified-id="Scrap-for-RedisList-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Scrap for RedisList</a></span></li></ul></li><li><span><a href="#Redis-types" data-toc-modified-id="Redis-types-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Redis types</a></span></li><li><span><a href="#Scrap" data-toc-modified-id="Scrap-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Scrap</a></span><ul class="toc-item"><li><span><a href="#Trying-redis-out" data-toc-modified-id="Trying-redis-out-6.1"><span class="toc-item-num">6.1&nbsp;&nbsp;</span>Trying redis out</a></span></li><li><span><a href="#RedisPersister" data-toc-modified-id="RedisPersister-6.2"><span class="toc-item-num">6.2&nbsp;&nbsp;</span>RedisPersister</a></span></li></ul></li></ul></div>

# Prerequisites

You'll need the `redisdol` package, which you can install as such:

```
    pip install redisdol
```

This will also install it's dependencies: `redis` and `dol`. 

What it will **not** install for you is the actual redis backend. 
To get/install Redis see here: https://redis.io/topics/quickstart

To run this notebook, you'll need to be running a redis server.

To launch local redis server, do this in your terminal:
```
    redis-server
```

# Redis in pystore

## Making a store

In [54]:
from py2store.persisters.redis_w_redis import (
    RedisList, RedisBytesPersister, RedisCollection, RedisPersister, Redis
)

Let's make a writer, for when we need to write. But limited to bytes writing at this point.

In [57]:
s = RedisPersister()
list(s)

[]

## The normal case is a string/bytes

In [58]:
k = '__test_string'
del s[k]  # note that Redis doesn't complain when you delete a key that's not there
s[k] = "This is not a string"
print(list(s))
assert s[k] == b'This is not a string'  # indeed it's not: It's a bytes

[b'__test_string']


## Lists

In [95]:
kk = '__test_list'
del s[kk]  # note that Redis doesn't complain when you delete a key that's not there
print(f"number of items: {len(s)}")
s[kk] = [1, 2, 3]
print(f"number of items: {len(s)}")
v = s[kk]
v

number of items: 3
number of items: 4


Note:

In [96]:
type(v)

redisdol.RedisList

What's that?

Well, it's a list-like object (but not exactly a list). 

To get a list, you can use `list(obj)`, as you may have guessed. 

In [97]:
list(v)

[b'1', b'2', b'3']

Yeah... that's not what you put there, but these are the base classes, so we stay close to the base library. 

Don't worry, you can have keys and values in what ever form you want, by using the right subclasses or decorators. But let's not go there yet: If you're curious, you can check out [`wrap_kvs`](https://i2mint.github.io/dol/module_docs/dol/trans.html#dol.trans.wrap_kvs) for example.

Meanwhile, if you need a more convenient store to handle lists of numbers automatically for you, try out `RedisStoreWithNumericLists`:

In [132]:
from redisdol.stores import RedisStoreWithNumericLists

ss = RedisStoreWithNumericLists()
ss['__test_numeric_list'] = [1,-2,-3.0, 4e-23, 'not_a_number']
ss['__test_numeric_list']

[1, -2, -3.0, 4e-23, b'not_a_number']

Back to the `RedisList` object...

The advantage of the store not returning a list, but a special list-like object, is when you write to this `RedisList` object, the writes are actually forwarded to the Redis persister. That wouldn't happen if you were given simply a (copy of the underlying) list. 

In [9]:
v.append(4)
list(v)

[b'1', b'2', b'3', b'4']

In [10]:
v.extend(['bob', 'and', 'alice'])
list(v)

[b'1', b'2', b'3', b'4', b'bob', b'and', b'alice']

In [11]:
v[4]  # you can get an item by index

b'bob'

In [12]:
v[4:7]  # you can slice (without step)

[b'bob', b'and', b'alice']

In [13]:
v[:3]  # you can also slice like this

[b'1', b'2', b'3']

In [14]:
v[:-1]  # or do this

[b'1', b'2', b'3', b'4', b'bob', b'and']

In [15]:
v[3]

b'4'

In [16]:
v[3] = 'not four anymore'  # oh, and you can write to an index

In [17]:
list(v)

[b'1', b'2', b'3', b'not four anymore', b'bob', b'and', b'alice']

Note that if you ask for an index that's not there, you just get `None`. 

That's also what you would get if you actually stored a None there, so perhaps not the best choice. 
But it's what `redis.Redis` chose, and we're just the messenger. 
Also, you can't store a `None`, so...

In [18]:
v[len(v)]  # nothing there.

In [19]:
v[len(v) + 100000]  # nothing there either.

In [20]:
len(v)

7

In [21]:
# v[len(v)] = 'why would you do this, you have append!'  # this does not work

In [22]:
# v[4242] = 'skipping a few entries, but I really want it there!'  # this does not work either

So you can update a `RedisList`, but writes are still destroying.

In [23]:
list(s[kk])

[b'1', b'2', b'3', b'not four anymore', b'bob', b'and', b'alice']

In [24]:
s[kk] = "poof, it's gone"
s[kk]

b"poof, it's gone"

# Using Redis to operate on streams

For the following, you'll need to `pip install meshed` if you don't have it already. 
In it, we'll be using `Slabs`, which is a class to define stream operations: Both where to source streams and what operations/services to run on these. 

In [34]:
from know.base import Slabs
from statistics import stdev

vol = stdev

# Making a slabs iter object
def make_a_slabs_iter(scope_factory=dict):

    # Mocking the sensor readers
    audio_sensor_read = iter([[1, 2, 3], [-96, 87, -92], [320, -96, 99]]).__next__
    light_sensor_read = iter([126, 501, 523]).__next__
    movement_sensor_read = iter([None, None, True]).__next__

    return Slabs(
        # The first three components get data from the sensors.
        # The *_read objects are all callable, returning the next
        # chunk of data for that sensor, if any.
        scope_factory=scope_factory,
        audio=audio_sensor_read,
        light=light_sensor_read,
        movement=movement_sensor_read,
        # The next
        should_turn_movement_sensor_on = lambda audio, light: vol(audio) * light > 50000,
        human_presence_score = lambda audio, light, movement: movement and sum([vol(audio), light]),
        should_notify = lambda human_presence_score: human_presence_score and human_presence_score > 700,
        notify = lambda should_notify: print('someone is there') if should_notify else None
    )


si = make_a_slabs_iter()
list(si)


someone is there


[{'audio': [1, 2, 3],
  'light': 126,
  'movement': None,
  'should_turn_movement_sensor_on': False,
  'human_presence_score': None,
  'should_notify': None,
  'notify': None},
 {'audio': [-96, 87, -92],
  'light': 501,
  'movement': None,
  'should_turn_movement_sensor_on': True,
  'human_presence_score': None,
  'should_notify': None,
  'notify': None},
 {'audio': [320, -96, 99],
  'light': 523,
  'movement': True,
  'should_turn_movement_sensor_on': True,
  'human_presence_score': 731.1353726143957,
  'should_notify': True,
  'notify': None}]

If we just feed a default RedisBytesPersister as the scope_factory, it will fail with a `DataError`.

In [135]:
si = make_a_slabs_iter(scope_factory=RedisPersister)
# list(si)
# raises
# DataError: Invalid input of type: 'list'. Convert to a bytes, string, int or float first.

We need to do a bit more work on the serialization part.

In [146]:
from redisdol.stores import RedisStoreWithNumericLists
si = make_a_slabs_iter(scope_factory=RedisStoreWithNumericLists)
# next(si)

In [143]:
t = RedisStoreWithNumericLists()
t['asdad'] = [1,2,3]

In [144]:
list(t['asdad'])

[1, 2, 3]

In [ ]:
RedisStoreWithNumericLists

# Ignore everything below -- it's just scrap

In [2]:
# Look at what the redis.Redis class contains:
from redis import Redis

print(*dir(Redis), sep='\t')

RESPONSE_CALLBACKS	__abstractmethods__	__annotations__	__class__	__class_getitem__	__contains__	__del__	__delattr__	__delitem__	__dict__	__dir__	__doc__	__enter__	__eq__	__exit__	__format__	__ge__	__getattribute__	__getitem__	__gt__	__hash__	__init__	__init_subclass__	__le__	__lt__	__module__	__ne__	__new__	__parameters__	__reduce__	__reduce_ex__	__repr__	__setattr__	__setitem__	__sizeof__	__slots__	__str__	__subclasshook__	__weakref__	_abc_impl	_disconnect_raise	_eval	_evalsha	_fcall	_georadiusgeneric	_geosearchgeneric	_is_protocol	_is_runtime_protocol	_send_command_parse_response	_zaggregate	_zrange	acl_cat	acl_deluser	acl_dryrun	acl_genpass	acl_getuser	acl_help	acl_list	acl_load	acl_log	acl_log_reset	acl_save	acl_setuser	acl_users	acl_whoami	append	auth	bf	bgrewriteaof	bgsave	bitcount	bitfield	bitfield_ro	bitop	bitpos	blmove	blmpop	blpop	brpop	brpoplpush	bzmpop	bzpopmax	bzpopmin	cf	client	client_getname	client_getredir	client_id	client_info	client_kill	client_kill_filter	client_list

## Scrap for RedisList

Say you wanted to make a reader, or other instance, using the same source. 
Consider the following methods.

In [63]:
# We could make it from scratch by doing RedisCollection(), but we could also reuse the same _source
r = RedisCollection.from_source(w._source)  # one option: Reusing w's _source attr
r = RedisCollection.from_sourced_object(w)  # another option: directly from w, that has a _source attr
list(r)

[b'one', b'__test_name', b'a list', b'bytes', b'int', b'float', b'foo']

See how RedisCollection reacts to different types

In [64]:
r['bytes']

b'blah'

... but if it's a list

In [44]:
t = r['another_list']

In [47]:
list(t)

[b'3', b'10', b'3', b'10', b'bob']

In [48]:
t.append('alice')
list(t)

[b'alice', b'3', b'10', b'3', b'10', b'bob']

In [50]:
t.extend(['a', 'b', 'c'])
list(t)

[b'c', b'b', b'a', b'alice', b'3', b'10', b'3', b'10', b'bob']

In [55]:
# del w['another_list']

In [6]:
s[name]

ResponseError: WRONGTYPE Operation against a key holding the wrong kind of value

In [78]:
name = 'another_list'
ss = RedisListReader(s._source, name)

In [80]:
len(ss)

5

In [81]:
list(ss)

[b'3', b'10', b'3', b'10', b'bob']

In [84]:
ss[1]

b'10'

In [89]:
s._source.lpush('a list', 1, 2, 3)

3

In [92]:
list(RedisListReader(s._source, 'a list'))

[b'3', b'2', b'1']

In [112]:
ss = RedisList(s._source, 'a list')

In [113]:
list(ss)

[b'3', b'2', b'1']

In [114]:
ss[1] = 'hello'
list(ss)

[b'3', b'hello', b'1']

In [115]:
ss = RedisList(s._source, 'does not exist')

In [ ]:
Redis.lpushx()

In [116]:
ss[0] = 'now'

ResponseError: no such key

In [ ]:
list.extend()

# Redis types

Taking the Type/Commands table from https://realpython.com/python-redis/#more-data-types-in-python-vs-redis, copy/paste/editing to csv, and using https://www.tablesgenerator.com/markdown_tables, I get:

| Type 	| Commands 	|
|-	|-	|
| Sets 	| SADD, SCARD, SDIFF, SDIFFSTORE, SINTER, SINTERSTORE, SISMEMBER, SMEMBERS, SMOVE, SPOP, SRANDMEMBER, SREM, SSCAN, SUNION, SUNIONSTORE 	|
| Hashes 	| HDEL, HEXISTS, HGET, HGETALL, HINCRBY, HINCRBYFLOAT, HKEYS, HLEN, HMGET, HMSET, HSCAN, HSET, HSETNX, HSTRLEN, HVALS" 	|
| Lists 	| BLPOP, BRPOP, BRPOPLPUSH, LINDEX, LINSERT, LLEN, LPOP, LPUSH, LPUSHX, LRANGE, LREM, LSET, LTRIM, RPOP, RPOPLPUSH, RPUSH, RPUSHX 	|
| Strings 	| APPEND, BITCOUNT, BITFIELD, BITOP, BITPOS, DECR, DECRBY, GET, GETBIT, GETRANGE, GETSET, INCR, INCRBY, INCRBYFLOAT, MGET, MSET, MSETNX, PSETEX, SET, SETBIT, SETEX, SETNX, SETRANGE, STRLEN 	||  

In [54]:
from redis import Redis
from inspect import signature

list_methods = [x.lower().strip() for x in 
                "BLPOP, BRPOP, BRPOPLPUSH, LINDEX, LINSERT, LLEN, LPOP, LPUSH, LPUSHX, LRANGE, \
                LREM, LSET, LTRIM, RPOP, RPOPLPUSH, RPUSH, RPUSHX".split(', ')]
not_found = []
for method in list_methods:
    if not method in dir(Redis):
        not_found.append(method)
    assert hasattr(Redis, method)
if not_found:
    print(f"Methods not found:\n{not_found}")
print("")
list_methods = list(set(list_methods) - set(not_found))

import re
space = re.compile('\s+$')
def get_first_doc_line(obj):
    for line in obj.__doc__.splitlines():
        if line and not space.match(line):
            return line.strip() + "..."
    return ""

for method in list_methods:
    m = getattr(Redis, method)
    print(f"{method}: {signature(m)}" + "\n\t" + f"{get_first_doc_line(m)}")


rpushx: (self, name, value)
	Push ``value`` onto the tail of the list ``name`` if ``name`` exists...
lset: (self, name, index, value)
	Set ``position`` of list ``name`` to ``value``...
brpop: (self, keys, timeout=0)
	RPOP a value off of the first non-empty list...
rpop: (self, name)
	Remove and return the last item of the list ``name``...
rpoplpush: (self, src, dst)
	RPOP a value off of the ``src`` list and atomically LPUSH it...
lrange: (self, name, start, end)
	Return a slice of the list ``name`` between...
blpop: (self, keys, timeout=0)
	LPOP a value off of the first non-empty list...
lpush: (self, name, *values)
	Push ``values`` onto the head of the list ``name``...
lrem: (self, name, count, value)
	Remove the first ``count`` occurrences of elements equal to ``value``...
ltrim: (self, name, start, end)
	Trim the list ``name``, removing all values not within the slice...
lpop: (self, name)
	Remove and return the first item of the list ``name``...
llen: (self, name)
	Return the leng

In [93]:
from collections import deque

In [95]:
dir(deque)

['__add__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'appendleft',
 'clear',
 'copy',
 'count',
 'extend',
 'extendleft',
 'index',
 'insert',
 'maxlen',
 'pop',
 'popleft',
 'remove',
 'reverse',
 'rotate']

In [104]:
t = deque([1,2,3], maxlen=3)
t.append(4)

In [109]:
t = [1,2,3,4]
t.insert(2, 'asdf')
t

[1, 2, 'asdf', 3, 4]

In [ ]:
Redis.linsert()

In [108]:
class A(deque):
    passma
    t

NameError: name 'passma' is not defined

# Scrap

## Trying redis out

In [121]:
import redis

r = redis.Redis()
# print(", ".join(dir(r)))

In [122]:
r.set('hello', 'world')
r.keys()

[b'hello', b'foo']

In [126]:
r.__setitem__('one', 'two')
r.keys()

[b'hello', b'one', b'foo']

In [127]:
'one' in r

True

In [ ]:
s['users'][10]

In [ ]:
s['users'][10]  r.get('users', 10)

In [ ]:
s['users'][10]

In [130]:
r.type('foo')

b'string'

In [133]:
type(r['foo'])

bytes

In [138]:
r.set('a_list', 1)
r.type('a_list')

b'string'

In [141]:
r.rpush('another_list', 3, 10, 'bob')

5

In [143]:
r.type('another_list')

b'list'

In [153]:
r.lrange('another_list', 0, int(1e3))

[b'3', b'10', b'3', b'10', b'bob']

In [135]:
r.type('bytes')

b'string'

In [132]:
import builtins
'input' in dir(builtins)

import inspect
assert not inspect.isbuiltin(builtins.list)
assert not inspect.isbuiltin(builtins.list())

In [99]:
r.set('foo', 'bar')

True

## RedisPersister

In [56]:
from py2store.persisters.redis_w_redis import RedisPersister

In [57]:
s = RedisPersister()

In [58]:
list(s)

[b'one', b'another_list', b'bytes', b'int', b'float', b'foo']

In [65]:
from itertools import count
for i in range(s._source.llen('another_list')):
    print(s._source.lindex('another_list', i))

b'3'
b'10'
b'3'
b'10'
b'bob'


In [163]:
del s['hello']
del s['a_list']

In [180]:
from py2store.persisters.redis_w_redis import RedisPersister
s = RedisPersister()  # plenty of params possible (all those of redis.Redis), but taking defaults. 

keys = ['_pyst_test_str', '_pyst_test_int', '_pyst_test_float']
for k in keys:
    del s[k]
    
    
before_length = len(s)


s['_pyst_test_str'] = 'hello'
assert s['_pyst_test_str'] == b'hello'

s['_pyst_test_int'] = 42
assert s['_pyst_test_int'] == b'42'

s['_pyst_test_float'] = 3.14
assert s['_pyst_test_float'] == b'3.14'

assert len(s) == before_length + 3

keys = ['_pyst_test_str', '_pyst_test_int', '_pyst_test_float']
for k in keys:
    del s[k]

In [47]:
space.match('     ')

<re.Match object; span=(0, 5), match='     '>

In [53]:
import re
space = re.compile('\s+$')
def get_first_doc_line(obj):
    for line in obj.__doc__.splitlines():
        if line and not space.match(line):
            return line.strip() + "..."
    return ""

get_first_doc_line(Redis.linsert)

'Insert ``value`` in list ``name`` either immediately before or after...'

In [44]:
len(Redis.linsert.__doc__.splitlines())

7

In [40]:
Redis.linsert.__doc__.splitlines()


['',
 '        Insert ``value`` in list ``name`` either immediately before or after',
 '        [``where``] ``refvalue``',
 '',
 '        Returns the new length of the list on success or -1 if ``refvalue``',
 '        is not in the list.',
 '        ']

In [186]:
[x for x in dir(s._source) if x.startswith('h')]

['hdel',
 'hexists',
 'hget',
 'hgetall',
 'hincrby',
 'hincrbyfloat',
 'hkeys',
 'hlen',
 'hmget',
 'hmset',
 'hscan',
 'hscan_iter',
 'hset',
 'hsetnx',
 'hstrlen',
 'hvals']

In [185]:
[x for x in dir(s._source) if x.startswith('l')]

['lastsave',
 'lindex',
 'linsert',
 'llen',
 'lock',
 'lpop',
 'lpush',
 'lpushx',
 'lrange',
 'lrem',
 'lset',
 'ltrim']

In [187]:
[x for x in dir(s._source) if x.startswith('s')]

['sadd',
 'save',
 'scan',
 'scan_iter',
 'scard',
 'script_exists',
 'script_flush',
 'script_kill',
 'script_load',
 'sdiff',
 'sdiffstore',
 'sentinel',
 'sentinel_get_master_addr_by_name',
 'sentinel_master',
 'sentinel_masters',
 'sentinel_monitor',
 'sentinel_remove',
 'sentinel_sentinels',
 'sentinel_set',
 'sentinel_slaves',
 'set',
 'set_response_callback',
 'setbit',
 'setex',
 'setnx',
 'setrange',
 'shutdown',
 'sinter',
 'sinterstore',
 'sismember',
 'slaveof',
 'slowlog_get',
 'slowlog_len',
 'slowlog_reset',
 'smembers',
 'smove',
 'sort',
 'spop',
 'srandmember',
 'srem',
 'sscan',
 'sscan_iter',
 'strlen',
 'substr',
 'sunion',
 'sunionstore',
 'swapdb']

In [188]:
[x for x in dir(s._source) if 'card' in x]

['scard', 'zcard']

In [ ]:
s._source.zcard()

In [190]:
s._source.set('boo', [1,2,3])

DataError: Invalid input of type: 'list'. Convert to a bytes, string, int or float first.

In [191]:
list(s)

[b'one', b'another_list', b'bytes', b'int', b'float', b'foo']

In [193]:
s._source.lrange('another_list', 0, 3)

[b'3', b'10', b'3', b'10']

In [201]:
s._source.llen('another_list')

5

In [199]:
signature(s._source.llen)

<Signature (name)>

In [203]:
from collections.abc import Sequence
set(dir(Sequence)) - set(dir(list))

{'__abstractmethods__', '__module__', '__slots__', '_abc_impl'}

In [204]:
set(dir(list)) - set(dir(Sequence))

{'__add__',
 '__delitem__',
 '__iadd__',
 '__imul__',
 '__mul__',
 '__rmul__',
 '__setitem__',
 'append',
 'clear',
 'copy',
 'extend',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort'}

In [66]:
t.stop = 

slice(None, 10, None)

In [ ]:
class RedisListReader(Sequence):
    def __init__(self, source, name):
        self._source = source
        self._name = name

    def __len__(self):
        return self._source.llen()

    def __getitem__(self, k):
        return self._source.lindex(self._name, k)

    def __iter__(self):
        # TODO: Find a more efficient way to do this.
        #   In batches perhaps? But what's the optimal batch size?
        #   async fetching of the next value/batch while yielding the one that's already fetched?
        for i in range(len(self)):
            yield self[i]
